In [1]:
from pathlib import Path
import pandas as pd
import os, json
from analysis_utils import get_tuned_alg_perf
root_dir = os.path.dirname(os.getcwd())

In [2]:
def read_json(file):
    with open(file, 'r') as f:
        return json.load(f)

In [3]:
metadata_folder = Path("../TabZilla")
metadataset_df = pd.read_csv(metadata_folder / "tutorials/metadataset_clean.csv")

In [4]:
metadataset_df.head()

,dataset_fold_id,dataset_name,target_type,alg_name,hparam_source,Log Loss__train,Log Loss__val,Log Loss__test,AUC__train,AUC__val,...,Accuracy__train,Accuracy__val,Accuracy__test,F1__train,F1__val,F1__test,training_time,eval-time__train,eval-time__val,eval-time__test
0,openml__APSFailure__168868__fold_0,openml__APSFailure__168868,binary,CatBoost,default,0.018501,0.026880,0.020194,0.990573,0.986760,...,0.994523,0.991053,0.993421,0.994523,0.991053,0.993421,4.039096,0.185535,0.028121,0.034166
1,openml__APSFailure__168868__fold_0,openml__APSFailure__168868,binary,CatBoost,random_1_s0,0.011968,0.024195,0.017324,0.995348,0.988589,...,0.997533,0.991974,0.994605,0.997533,0.991974,0.994605,3.624039,0.129900,0.020647,0.017871
2,openml__APSFailure__168868__fold_0,openml__APSFailure__168868,binary,CatBoost,random_10_s0,0.022589,0.029668,0.023737,0.990179,0.985547,...,0.992566,0.989605,0.990921,0.992566,0.989605,0.990921,2.494685,0.074099,0.021543,0.019580
3,openml__APSFailure__168868__fold_0,openml__APSFailure__168868,binary,CatBoost,random_11_s0,0.028515,0.034531,0.028539,0.987448,0.985578,...,0.990641,0.986974,0.990132,0.990641,0.986974,0.990132,2.532277,0.061874,0.014086,0.013869
4,openml__APSFailure__168868__fold_0,openml__APSFailure__168868,binary,CatBoost,random_12_s0,0.015119,0.025570,0.019329,0.993757,0.989098,...,0.996382,0.991447,0.993684,0.996382,0.991447,0.993684,4.220030,0.121606,0.034948,0.033927


In [5]:


new_df = []

for model in ['TabPFNModel', 'TabFlexModel', 'TabFastModel', 'HyperFastModel', 'TunetablesModel', 'TabSmallModel']:
    datasets = os.listdir(f'../results/{model}')
    for dataset in datasets:
        if model in ['TabFastModel', 'TabFlexModel', 'HyperFastModel', 'TunetablesModel']:
            try:
                result = read_json(f'../results/{model}/{dataset}/1000000000000000/default_trial0_results.json')
                if not 'train' in result['scorers']: raise Exception('No train scorers')
            except:
                result = read_json(f'../results/{model}/{dataset}/100000/default_trial0_results.json')
                
        elif model in ['TabPFNModel', 'TabSmallModel']:
            result = read_json(f'../results/{model}/{dataset}/3000/default_trial0_results.json')
        
            
        for fold_id in range(10):
            try:
                new_df.append({
                    'dataset_fold_id': f'{dataset}__fold_{fold_id}',
                    'dataset_name': dataset,
                    'target_type': result['dataset']['target_type'],
                    'alg_name': model,
                    'hparam_source': result['hparam_source'],
                    'Log Loss__train': result['scorers']['train']['Log Loss'][fold_id],
                    'Log Loss__val': result['scorers']['val']['Log Loss'][fold_id],
                    'Log Loss__test': result['scorers']['test']['Log Loss'][fold_id],
                    'AUC__train': result['scorers']['train']['AUC'][fold_id],
                    'AUC__val': result['scorers']['val']['AUC'][fold_id],
                    'AUC__test': result['scorers']['test']['AUC'][fold_id],
                    'Accuracy__train': result['scorers']['train']['Accuracy'][fold_id],
                    'Accuracy__val': result['scorers']['val']['Accuracy'][fold_id],
                    'Accuracy__test': result['scorers']['test']['Accuracy'][fold_id],
                    'F1__train': result['scorers']['train']['F1'][fold_id],
                    'F1__val': result['scorers']['val']['F1'][fold_id],
                    'F1__test': result['scorers']['test']['F1'][fold_id],
                    'training_time': result['timers']['train'][1] if fold_id == 0 else result['timers']['train'][fold_id],
                    'time__val': result['timers']['val'][fold_id],
                    'time__test': result['timers']['test'][fold_id],
                    'eval-time__train': result['timers']['train-eval'][fold_id],
                    'eval-time__val': result['timers']['val'][fold_id],
                    'eval-time__test': result['timers']['test'][fold_id],
                })
            except Exception as e:
                print(e)
                print(f'{model}:Error with {dataset} fold {fold_id}')
        
new_df = pd.DataFrame(new_df)

'train'
HyperFastModel:Error with openml__MiniBooNE__168335 fold 0
'train'
HyperFastModel:Error with openml__MiniBooNE__168335 fold 1
'train'
HyperFastModel:Error with openml__MiniBooNE__168335 fold 2
'train'
HyperFastModel:Error with openml__MiniBooNE__168335 fold 3
'train'
HyperFastModel:Error with openml__MiniBooNE__168335 fold 4
'train'
HyperFastModel:Error with openml__MiniBooNE__168335 fold 5
'train'
HyperFastModel:Error with openml__MiniBooNE__168335 fold 6
'train'
HyperFastModel:Error with openml__MiniBooNE__168335 fold 7
'train'
HyperFastModel:Error with openml__MiniBooNE__168335 fold 8
'train'
HyperFastModel:Error with openml__MiniBooNE__168335 fold 9
'train'
HyperFastModel:Error with openml__poker-hand__9890 fold 0
'train'
HyperFastModel:Error with openml__poker-hand__9890 fold 1
'train'
HyperFastModel:Error with openml__poker-hand__9890 fold 2
'train'
HyperFastModel:Error with openml__poker-hand__9890 fold 3
'train'
HyperFastModel:Error with openml__poker-hand__9890 fold 4


In [6]:
# concatenate metadataset_df and new_df
updated_df = pd.concat([metadataset_df, new_df], axis=0)

In [7]:
def get_dataset_stat(dataset):
    result_dir = f"{root_dir}/results/TabFlexModel/{dataset}"
    if '100000' in os.listdir(result_dir):
        result_dir = f"{result_dir}/100000"
    else:
        result_dir = f"{result_dir}/1000000000000000"
    items = os.listdir(result_dir)
    for item in items:
        if item == 'default_trial0_results.json':
            result = read_json(f"{result_dir}/{item}")
            break
        elif os.path.isdir(f"{result_dir}/{item}"):
            result = read_json(f"{result_dir}/{item}/default_trial0_results.json")
            break
    return {
        'num_classes': result['dataset']['num_classes'],
        'num_features': result['dataset']['num_features'],
        'num_instances': result['dataset']['num_instances'],
    }

In [8]:
len(updated_df[updated_df.alg_name == 'TabFlexModel'].dataset_fold_id.unique())//10

115

In [9]:
new_rows = []
# iterate over the datasets 
dataset_fold_ids = updated_df[updated_df.alg_name == 'TabFlexModel'].dataset_fold_id.unique()
for dataset_fold_id in dataset_fold_ids:
    tabpfn_row = updated_df[(updated_df.dataset_fold_id == dataset_fold_id) & (updated_df.alg_name == 'TabPFNModel')].iloc[-1]
    tabflex_row = updated_df[(updated_df.dataset_fold_id == dataset_fold_id) & (updated_df.alg_name == 'TabFlexModel')].iloc[-1]
    tabfast_row = updated_df[(updated_df.dataset_fold_id == dataset_fold_id) & (updated_df.alg_name == 'TabFastModel')].iloc[-1]
    tabsmall_row = updated_df[(updated_df.dataset_fold_id == dataset_fold_id) & (updated_df.alg_name == 'TabSmallModel')].iloc[-1]
    data_stat = get_dataset_stat(tabpfn_row.dataset_name)
    if data_stat['num_instances'] >= 3000 and data_stat['num_features'] <= 100:
        new_row = tabfast_row.copy()
    elif data_stat['num_features'] > 100 or ((data_stat['num_features'] / data_stat['num_instances'] >= 0.2) and data_stat['num_instances'] >= 3000):
        new_row = tabflex_row.copy()
    else:
        new_row = tabpfn_row.copy()
        
    new_row['alg_name'] = 'Ours-1'
    new_rows.append(new_row)
    
    if data_stat['num_instances'] >= 3000 and data_stat['num_features'] <= 100:
        new_row = tabfast_row.copy()
    elif data_stat['num_features'] > 100 or ((data_stat['num_features'] / data_stat['num_instances'] >= 0.2) and data_stat['num_instances'] >= 3000):
        new_row = tabflex_row.copy()
    else:
        new_row = tabsmall_row.copy()
    
    new_row['alg_name'] = 'Ours-2'
    new_rows.append(new_row)
    
    

In [10]:
updated_df[(updated_df.alg_name == 'TabFastModel')]

,dataset_fold_id,dataset_name,target_type,alg_name,hparam_source,Log Loss__train,Log Loss__val,Log Loss__test,AUC__train,AUC__val,...,Accuracy__test,F1__train,F1__val,F1__test,training_time,eval-time__train,eval-time__val,eval-time__test,time__val,time__test
2300,openml__MiniBooNE__168335__fold_0,openml__MiniBooNE__168335,binary,TabFastModel,default,0.201090,0.209462,0.203652,0.974844,0.972350,...,0.917890,0.917299,0.913739,0.917890,0.035624,11.688162,6.321706,8.127416,6.321706,8.127416
2301,openml__MiniBooNE__168335__fold_1,openml__MiniBooNE__168335,binary,TabFastModel,default,0.201540,0.211238,0.209050,0.974782,0.972304,...,0.912970,0.916944,0.912816,0.912970,0.035624,10.717506,8.620559,7.441536,8.620559,7.441536
2302,openml__MiniBooNE__168335__fold_2,openml__MiniBooNE__168335,binary,TabFastModel,default,0.203508,0.207199,0.212483,0.974552,0.974106,...,0.911894,0.915925,0.913893,0.911894,0.035598,7.091975,6.165642,6.178733,6.165642,6.178733
2303,openml__MiniBooNE__168335__fold_3,openml__MiniBooNE__168335,binary,TabFastModel,default,0.204547,0.199211,0.206274,0.974248,0.975158,...,0.914738,0.915445,0.919114,0.914738,0.033125,8.195157,8.456621,8.191070,8.456621,8.191070
2304,openml__MiniBooNE__168335__fold_4,openml__MiniBooNE__168335,binary,TabFastModel,default,0.204402,0.209648,0.198884,0.974278,0.973027,...,0.918268,0.915437,0.911964,0.918268,0.035263,7.137135,7.273583,7.435564,7.273583,7.435564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3445,openml__mfeat-factors__12__fold_5,openml__mfeat-factors__12,classification,TabFastModel,default,0.154391,0.279641,0.345398,0.997943,0.995583,...,0.910000,0.946163,0.892488,0.910187,0.000511,0.068474,0.066282,0.064238,0.066282,0.064238
3446,openml__mfeat-factors__12__fold_6,openml__mfeat-factors__12,classification,TabFastModel,default,0.163608,0.185602,0.265902,0.997822,0.996833,...,0.905000,0.936506,0.954840,0.906608,0.000477,0.062878,0.058797,0.056004,0.058797,0.056004
3447,openml__mfeat-factors__12__fold_7,openml__mfeat-factors__12,classification,TabFastModel,default,0.171668,0.276521,0.193979,0.997679,0.996250,...,0.940000,0.939724,0.915823,0.939988,0.000429,0.087473,0.077365,0.065587,0.077365,0.065587
3448,openml__mfeat-factors__12__fold_8,openml__mfeat-factors__12,classification,TabFastModel,default,0.171980,0.327084,0.317405,0.997745,0.993694,...,0.905000,0.935312,0.905345,0.905450,0.000599,0.071302,0.064981,0.065187,0.064981,0.065187


In [11]:
new_df = pd.DataFrame(new_rows)
updated_df = pd.concat([updated_df, new_df], axis=0)
updated_df.to_csv(metadata_folder / "tutorials/metadataset_new.csv", index=False)